In [1]:
import time
import datetime as dt
import pandas as pd
import snowflake.connector
from pypardot.client import PardotAPI, PardotAPIError
import snowflake.connector
import requests
pd.options.display.float_format = '{:,.2f}'.format


In [2]:
PARDOT_EMAIL="segment_integrations@discoveryed.com"
PARDOT_USER_KEY="f5a1dc61d079e35d9a2066a4a8498c32"
PARDOT_SF_CONSUMER_KEY="3MVG9IHf89I1t8hpom1l0QzbTANHH.MOASIZ3yPPhu6hFI_uipXiYi7ku85yMWg_2gXxu5bzAyWCevOHp4jrf"
PARDOT_SF_CONSUMER_SECRET="BC6C2AA5D41DC0EBBBB3376CD329E92E66BD8D85DF85F9FAB9F22E86D857A29E"
PARDOT_SF_REFRESH_TOKEN="5Aep8618yVsldz6rZPMv4ouelrGoRNAfdaLSjo3.ILw4jdNGemJVI_MNYFoNkN9g39GUXqUEgzw3ylEVMjxAU3X"
PARDOT_BUSINESS_UNIT_ID="0Uv4P000000TNB0SAO"
PARDOT_API_VERSION=4
PARDOT_MAX_RESULT_COUNT = 200
SNOWFLAKE_ACCOUNT_IDENTIFIER="UK29315.us-east-1"
SNOWFLAKE_USER="PardotEtlUser@discoveryed.com"
SNOWFLAKE_PASS="p@rd0tEtlUser"
SF_SCHEMA = "PROD_DATA_VAULT"

In [3]:
def get_client_pardot() -> PardotAPI:
    try:
        p = PardotAPI(
            email=PARDOT_EMAIL,
            user_key=PARDOT_USER_KEY,
            sf_consumer_key=PARDOT_SF_CONSUMER_KEY,
            sf_consumer_secret=PARDOT_SF_CONSUMER_SECRET,
            sf_refresh_token=PARDOT_SF_REFRESH_TOKEN,
            business_unit_id=PARDOT_BUSINESS_UNIT_ID,
            version=PARDOT_API_VERSION,
        )

        if not p.authenticate():
            p.campaigns.query(limit=1)
            # ^ Must call query to authenticate if authenticatation is unsuccessful

    except PardotAPIError as err:
        raise

    assert (
        p.sftoken != "dummy"
    ), "Pardot authentication still unsuccessful after querying"

    return p

def get_client_snowflake():
    return snowflake.connector.connect(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASS,
        account=SNOWFLAKE_ACCOUNT_IDENTIFIER,
        # authenticator="externalbrowser",
        warehouse="GENERAL_COMPUTE_WH",
        database="PROD_DATA_VAULT",
        schema="STAGE",
    )

def query_sf(query_str):
    ctx = get_client_snowflake()
    cs = ctx.cursor()
    cs.execute(query_str)
    data =  cs.fetchall()  
    columns = [f'{col[0]}' for col in cs.description]
    return pd.DataFrame(data,columns=columns)

In [4]:
p = get_client_pardot()

In [5]:
tables = [
# "ListMembership",
'Campaign',
'Form',
# "VisitorActivity",
'Tag',
'Opportunity',
'EmailClick',
'List',
'Account',
'TagObject',
"Prospect",
"Visitor",
"ProspectAccount",
]

tag_object_types = ["Email","Campaign","List"]

DICT_CONVERSION_PURAL = {
    "Account": "accounts",
    "EmailClick": "emailclicks",
    "Email": "emails",
    "VisitorActivity": "visitoractivities",
    "Campaign": "campaigns",
    "List": "lists",
    "Prospect": "prospects",
    "TagObject": "tagobjects",
    "Tag": "tags",
    "Opportunity": "opportunities",
    "ProspectAccounts": "prospectaccounts",
    "Form": "forms",
    "ListMembership": "listmemberships",
    "Visitor": "visitors",
    "ProspectAccount": "prospectaccounts",
}


In [6]:
def helper_to_camelCase(s: str) -> str:
    "Return the same string with the first letter lowercase"
    return str(s[0].lower() + s[1:])

In [7]:
expected_results = {}
for data_type in tables:
    try:
        print(data_type)
        data_client = getattr(p, DICT_CONVERSION_PURAL[data_type])
        if not hasattr(data_client, "query"):
            # If the data_client doesn't have a query method, try the "read" method
            data = data_client.read()[helper_to_camelCase(data_type)]
            if isinstance(data, dict):
                # Assume only one record
                expected_results[data_type]=1
            elif isinstance(data, list):
                expected_results[data_type]=len(data)
        else:
            if data_type=="TagObject":
                counter = 0
                for tag_type in tag_object_types:
                    time.sleep(2)
                    data_raw = data_client.query(limit=1,type=tag_type)
                    counter += data_raw["total_results"]
                expected_results[data_type] = counter
            else:
                data_raw = data_client.query(limit=1)
                expected_results[data_type]=data_raw["total_results"]
    except:
        print(f"{data_type} failed")
        raise



Campaign
Form
Tag
Opportunity
EmailClick
List
Account
TagObject
Prospect
Visitor
ProspectAccount


In [8]:
tables = ['Campaign',
'Form',
'Tag',
'Opportunity',
'EmailClick',
'List',
'Account',
'TagObject',
"Email",
"EmailStat",
"Prospect",
"VisitorActivity",
"Visitor",
"ListMembership",
"ProspectAccount"
]

queries = []
for table in tables:
    if table == "TagObject":
        sql_template = f"""select a.data_object,count_market_distinct,count_market_total,count_stage_distinct,count_stage_total from (
        select '{table}' as data_object,count(distinct tag_object_id) as count_market_distinct, count(1) as count_market_total from prod_data_vault.marketing.{table}) a
        left join (select '{table}' as data_object,count(distinct tag_object_id) as count_stage_distinct, count(1) as count_stage_total from prod_data_vault.stage.PARDOT_{table}) b
        on a.data_object = b.data_object"""
    elif table == "VisitorActivity":
        sql_template = f"""select a.data_object,count_market_distinct,count_market_total,count_stage_distinct,count_stage_total from (
        select '{table}' as data_object,count(distinct visitor_activity_id) as count_market_distinct, count(1) as count_market_total from prod_data_vault.marketing.{table}) a
        left join (select '{table}' as data_object,count(distinct visitor_activity_id) as count_stage_distinct, count(1) as count_stage_total from prod_data_vault.stage.PARDOT_{table}) b
        on a.data_object = b.data_object"""
    elif table=='EmailStat':
        sql_template = f"""select a.data_object,count_market_distinct,count_market_total,count_stage_distinct,count_stage_total from (
        select '{table}' as data_object,count(distinct list_email_id) as count_market_distinct, count(1) as count_market_total from prod_data_vault.marketing.{table}) a
        left join (select '{table}' as data_object,count(distinct list_email_id) as count_stage_distinct, count(1) as count_stage_total from prod_data_vault.stage.PARDOT_{table}) b
        on a.data_object = b.data_object"""
    else:
        sql_template = f"""select a.data_object,count_market_distinct,count_market_total,count_stage_distinct,count_stage_total from (
        select '{table}' as data_object,count(distinct {table}_id) as count_market_distinct, count(1) as count_market_total from prod_data_vault.marketing.{table}) a
        left join (select '{table}' as data_object,count(distinct {table}_id) as count_stage_distinct, count(1) as count_stage_total from prod_data_vault.stage.PARDOT_{table}) b
        on a.data_object = b.data_object"""
    queries.append(sql_template)

query_join = ' union all '.join(queries)

email_query = """select 
        count(distinct a.list_email_id) as expected
        from PARDOT_VISITORACTIVITY a
        where a.email_id is not null and a.list_email_id is not null"""

sf_data = query_sf(query_join)

try:
    ctx = get_client_snowflake()
    cs = ctx.cursor()
    cs.execute(email_query)
    email_expected =  cs.fetchone()[0]
finally:
    cs.close()
ctx.close()

# Add expected count for email object
sf_data.loc[sf_data["DATA_OBJECT"]=="Email","EXPECTED"] = email_expected
sf_data.loc[sf_data["DATA_OBJECT"]=="EmailStat","EXPECTED"] = email_expected

In [9]:
# Add expected count for all the other data objects
for d_obj, exected in expected_results.items():
    sf_data.loc[sf_data["DATA_OBJECT"]==d_obj,"EXPECTED"] = exected

# Determine discrepancy between expected and whats in stage
sf_data["DIFFERENCE"] = sf_data["EXPECTED"]-sf_data["COUNT_STAGE_DISTINCT"]

# Flag issues with # of records compared to whats expected to come out of Pardot
# It's almost never going to be perfect, but just check that it's relatively close
sf_data.loc[sf_data["EXPECTED"]<sf_data["COUNT_STAGE_DISTINCT"],"EXPECT_ISSUE"] = "EXPECTED LOWER"
sf_data.loc[sf_data["EXPECTED"]>sf_data["COUNT_STAGE_DISTINCT"],"EXPECT_ISSUE"] = "DATA MISSING"
# sf_data.loc[sf_data["EXPECTED"]==sf_data["COUNT_STAGE_DISTINCT"],"EXPECT_ISSUE"] = "CORRECT"
# Flag issues with duplicates in marketing schema (this would indicate a snowflake stored procedure error)
sf_data.loc[sf_data["COUNT_MARKET_DISTINCT"]<sf_data["COUNT_MARKET_TOTAL"],"MARKET_ISSUE"] = "DUPLICATES IN MARKET"
sf_data.loc[sf_data["COUNT_MARKET_DISTINCT"]!=sf_data["COUNT_STAGE_DISTINCT"],"STORED_PROC_ISSUE"] = "DIFFERENT THAN STAGE"

sf_data.fillna('')

# DATA OBJECT - which dataset this refers to
# COUNT_MARKET_DISTINCT - how many distinct IDs are in the marketing schema table
# COUNT_MARKET_TOTAL - how many total records are in the marketing schema table (this is used to check for duplicates against COUNT_MARKET_DISTINCT)
# COUNT_STAGE_DISTINCT - how many distinct IDs are in the stage schema table (this is compared to COUNT_MARKET_DISTINCT to determine if all records from stage have been synced to marketing)
# COUNT_STAGE_TOTAL - how many total records are in the stage schema table (there will be duplicates, but keep an eye on it for exponential growth. Signals issues with process)
# EXPECTED - This is how many records for each dataset currently exist in pardot. We expected COUNT_STAGE_DISTINCT to be very close to this number (visitor accuracy is an outstanding question)
# DIFFERENCE - Difference between EXPECTED and COUNT_STAGE_DISTINCT. Negative numbers mean data is in the warehouse that potentially shouldn't be. Positive means we are missing data. Just need to check it's close
# EXPECT_ISSUE - Flags if data is missing or if extra data exists
# MARKET_ISSUE - Checks for duplicate IDs in the marketing schema

,DATA_OBJECT,COUNT_MARKET_DISTINCT,COUNT_MARKET_TOTAL,COUNT_STAGE_DISTINCT,COUNT_STAGE_TOTAL,EXPECTED,DIFFERENCE,EXPECT_ISSUE,MARKET_ISSUE,STORED_PROC_ISSUE
0,Campaign,589,589,589,19901,589.00,0.00,,,
1,Form,14,14,14,119,12.00,-2.00,EXPECTED LOWER,,
2,Tag,190,190,190,253,177.00,-13.00,EXPECTED LOWER,,
3,Opportunity,8900,8900,8900,44507,"8,903.00",3.00,DATA MISSING,,
4,EmailClick,1231943,1231943,1231943,1305427,"1,232,092.00",149.00,DATA MISSING,,
5,List,1897,1897,1897,2088,"1,892.00",-5.00,EXPECTED LOWER,,
6,Account,1,1,1,35,1.00,0.00,,,
7,TagObject,4065,4065,4065,4960,"4,209.00",144.00,DATA MISSING,,
8,Email,8391,8391,8391,8391,"8,391.00",0.00,,,
9,EmailStat,8391,8391,8391,41435,"8,391.00",0.00,,,


In [10]:
# CHECK ON THE STATUS OF A BULK EXPORT (USE THE ID LISTED IN THE CLOUDWATCH LOGS)
headers = {
    "content-type": "application/json",
    "Authorization": f"Bearer {p.sftoken}",
    "Pardot-Business-Unit-Id": PARDOT_BUSINESS_UNIT_ID,
}
PARDOT_API_VERSION = 4
ID = 4522 #update this
PARDOT_URL_API = f"https://pi.pardot.com/api/export/version/{PARDOT_API_VERSION}/do/read/id/{ID}"

response = requests.get(
    PARDOT_URL_API,
            headers=headers,
            params=(("format", "json"),),
        )

response.json()["export"]["state"]
 

KeyError: 'export'

In [25]:
# Query for emailstats by list_email_id
import json

headers = {
    "content-type": "application/json",
    "Authorization": f"Bearer {p.sftoken}",
    "Pardot-Business-Unit-Id": PARDOT_BUSINESS_UNIT_ID,
}

PARDOT_API_VERSION = 4

updated_after = "2022-03-01"
updated_before = "2022-03-02"
delete_flag = True

# PARDOT_URL_API = f"https://pi.pardot.com/api/prospect/version/{PARDOT_API_VERSION}"
PARDOT_URL_API = f"https://pi.pardot.com/api/export/version/{PARDOT_API_VERSION}"

response_create = requests.post(
            f"{PARDOT_URL_API}/do/create",
            headers=headers,
            params=(("format", "json"),),
            data=json.dumps(
                {
                    "object": data_type,
                    "procedure": {
                        "name": "filter_by_updated_at",
                        "arguments": {
                            "updated_after": updated_after,
                            "updated_before": updated_before,
                            **(dict(deleted=True) if delete_flag == True else {})
                        },
                    },
                }
            ),
        )

response_create.json()

# - 4,692,616 million active
# - 5,582,408 million delete

# Default is active 4692616
 

{'@attributes': {'stat': 'ok', 'version': 1},
 'export': {'id': 5206,
  'state': 'Waiting',
  'object': 'prospectAccount',
  'fields': ['id',
   'name',
   'salesforce_id',
   'created_at',
   'updated_at',
   'created_by_id',
   'updated_by_id',
   'assigned_to_id',
   'annual_revenue',
   'billing_address_one',
   'billing_address_two',
   'billing_city',
   'billing_country',
   'billing_state',
   'billing_zip',
   'description',
   'employees',
   'fax',
   'industry',
   'number',
   'ownership',
   'phone',
   'rating',
   'shipping_address_one',
   'shipping_address_two',
   'shipping_city',
   'shipping_country',
   'shipping_state',
   'shipping_zip',
   'sic',
   'site',
   'ticker_symbol',
   'type',
   'website',
   'is_archived'],
  'procedure': {'name': 'filter_by_updated_at',
   'arguments': {'updated_after': '2022-03-01 00:00:00',
    'updated_before': '2022-03-02 00:00:00',
    'deleted': True}},
  'isExpired': False,
  'createdAt': '2022-03-09 12:48:34',
  'updatedAt

In [27]:
response_status = requests.get(
                    f"{PARDOT_URL_API}/do/read/id/{response_create.json()['export']['id']}",
                    headers=headers,
                    params=(("format", "json"),),
                )
response_status.json()["export"]["state"]

'Waiting'